In [ ]:
import lxml.html
import lxml.etree
import requests
import pandas as pd
import grequests

In [ ]:
columns = ['Регион', 'ТИК', 'УИК', 'Число избирателей, включенных в список избирателей ',
 'Число избирательных бюллетеней, полученных участковой избирательной комиссией',
 'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно',
 'Число избирательных бюллетеней, выданных в помещении для голосования в день голосования',
 'Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования',
 'Число погашенных избирательных бюллетеней',
 'Число избирательных бюллетеней в переносных ящиках для голосования',
 'Число бюллетеней в стационарных ящиках для голосования',
 'Число недействительных избирательных бюллетеней',
 'Число действительных избирательных бюллетеней',
 'Число утраченных избирательных бюллетеней',
 'Число избирательных бюллетеней, не учтенных при получении ',
 'Бабурин Сергей Николаевич',
 'Грудинин Павел Николаевич',
 'Жириновский Владимир Вольфович',
 'Путин Владимир Владимирович',
 'Собчак Ксения Анатольевна',
 'Сурайкин Максим Александрович',
 'Титов Борис Юрьевич',
 'Явлинский Григорий Алексеевич']



In [ ]:
result = []

In [ ]:
def parse_table(uik_table, result, region_name, tik_name):
    for i in range(len(uik_table[0])):
            result.append([region_name, tik_name] + [ x[i][0][0].text if k != 0 else x[i][0].text[x[i][0].text.rfind('№') + 1:] for k, x in enumerate(uik_table) if (k != 13)])

In [ ]:
def parse_tik_responses(tik_responses, result, region_name, tik_names):
    rs = [ grequests.get(lxml.html.fromstring(x.text).xpath('//a[text()="сайт избирательной комиссии субъекта Российской Федерации"]')[0].attrib['href']) for  x in tik_responses]
    uik_responses = grequests.map(rs)
    uik_tables = [lxml.html.fromstring(x.text).xpath('//table[@style="width:100%;overflow:scroll"]')[0] for x in uik_responses]
    for j in range(len(uik_tables)):
        parse_table(uik_tables[j], result, region_name, tik_names[j])

In [ ]:
response = requests.get('http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227')
tree = lxml.html.fromstring(response.text)
tr = tree.xpath('//table[@style="width:100%;overflow:scroll"]')[0][0]

region_names = [x[0][0].text for x in tr]
rs = (grequests.get(x[0][0].attrib['href']) for x  in tr)
region_responses = grequests.map(rs)

tik_trs = [lxml.html.fromstring(x.text).xpath('//table[@style="width:100%;overflow:scroll"]')[0][0] for x in region_responses]

for k, tik_tr in enumerate(tik_trs):
    tik_names = [x[0][0].text for x in tik_tr]
    rs = (grequests.get(x[0][0].attrib['href']) for  x in tik_tr)
    tik_responses = grequests.map(rs)
    
    parse_tik_responses(tik_responses, result, region_names[k], tik_names)
    
parse_tik_responses([region_responses[-2]], result, region_names[-2], [region_names[-2]])
parse_tik_responses([region_responses[-1]], result, region_names[-1], [region_names[-1]])

In [ ]:
data = pd.DataFrame(result, columns= columns)

In [ ]:
data.to_csv('vibory_data.csv', index= None)